In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import os
import traci
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
#sumo_gui_bin = '/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui'
simulConfig = os.getenv("SIMUL-CONFIG2")


print(sumo_gui_bin)

/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui


In [72]:
from state import get_state, queue
from reward import calculate_reward

In [73]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [184]  # == env.observation_space.shape
n_outputs = 16  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(128,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [74]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:

        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(Q_values)
        return Q_values.argmax()  # optimal action according to the DQN

In [75]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [76]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [77]:
replay_buffer = deque(maxlen=10000)
epoch = 220
nb_step = 80000



# PERIODE DE STABILITE

#traffic_scale = [0.1] * 12
# AUGMENTATION DE TRAFFIC
#traffic_scale.extend(np.linspace(0.1, 0.3, 100, dtype=float))
# OR
traffic_scale = 0.2



for episode in range(epoch):
    #SEED & TRAFFIC SCALING
    seed_value = np.random.randint(1000)
    #seed_value = 41



    sumoCmd = [sumo_bin, "-c",
               simulConfig,
               "--start",
               "--seed", str(seed_value),
               '--no-warnings',
               '--scale', str(np.random.uniform(0.1, 0.3))]




    if traci.isLoaded():
        try:
            traci.close()
        except traci.TraCIException:
            pass



    traci.start(sumoCmd)



    lane_ids =  traci.lane.getIDList()
    trafic_light_ids = traci.trafficlight.getIDList()
    #print(trafic_light_ids)

    state=np.array(get_state(lane_ids))


    #print(f"state : {state} lenght of state: {len(state)}")



    action=-1


    for step in range(nb_step):
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:

            ##Calcul de la reward
            next_state = np.array(get_state(lane_ids))
            reward = calculate_reward(state, next_state)

            replay_buffer.append((state, action, reward, next_state))

            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            feu_1 = action // 4
            feu_2 = action % 4


            traci.trafficlight.setPhase(trafic_light_ids[0],2*feu_1)
            traci.trafficlight.setPhase(trafic_light_ids[1],2*feu_2)
            #print("feu_1",traci.trafficlight.getPhase(trafic_light_ids[0]))
            #print("feu_2",traci.trafficlight.getPhase(trafic_light_ids[1]))

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


Step #720.00 (0ms ~= 10.00*RT, ~14000.00UPS, TraCI: 0ms, vehicles TOT 162 ACT 14 BUF 0)   episode : 0
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 178 ACT 12 BUF 0)                  
 Retrying in 1 seconds


[-0.1916454  -0.24423039 -0.46982422  0.29061705 -0.2899006   0.14097495
 -0.1845224   0.3648813   0.07129044 -0.12027151  0.24254     0.17562181
  0.10453715 -0.14251155  0.15539056 -0.18330508]
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 105 ACT 16 BUF 0)                  episode : 1
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 118 ACT 13 BUF 0)                  
 Retrying in 1 seconds


[-0.6465067  -0.22352254 -0.6751183   0.70680475 -0.37425017  0.62674123
 -0.43676034  0.5899184   0.37376827 -0.55452657  0.83327204  0.14318804
  0.38967952 -0.5901138  -0.16742742  0.1866833 ]
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 255 ACT 25 BUF 0)                  episode : 2
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 286 ACT 30 BUF 0)                  
 Retrying in 1 seconds


[-0.5379221  -0.59967905 -0.3263112   0.32766825 -0.64200354  0.8850155
 -0.5782311   0.42219785 -0.0281148  -0.48448193  1.3280371   0.27891067
  0.49193117 -0.2551995   0.46597692 -0.06193286]
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 96 ACT 13 BUF 0)                   episode : 3
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 105 ACT 15 BUF 0)                  
 Retrying in 1 seconds


...

[ 7.445795  17.190535   7.1338797  7.793293  10.615789  10.412127
 11.064914  11.664621  10.976639  13.482492   6.7951574  5.749386
 10.13975    8.482618   1.3313069  6.6136055]
[ 5.9968967 12.736675   5.1392527  1.03883    6.1669497  6.199434
 11.480966  11.024922   9.36046    6.911313   2.9978302  4.8586864
  8.069266   5.28733    7.3048196  1.9123439]
[ 7.3819084 12.8744135  4.3596067  1.3705645  6.752068   8.884226
  9.443836  11.566242  10.226913   7.2279344  5.382202   5.439562
  8.616011   6.2259817  8.098698   2.9753313]
[ 9.543394  14.272326   6.9654765  4.1539564  7.662897  15.636375
 10.7023945 13.141158  12.455456   8.375413   8.813723   6.5162964
 10.332645   8.93226    7.6981773  4.0245314]
[12.409752  14.592825   8.070338   6.28131    8.517237  22.521414
  9.831615  14.594863  13.675531   7.466572  12.659206   6.3768983
 11.498142  13.138578  10.990499   3.6760516]
[12.262274  13.499832  17.418793  10.406063   6.451392  23.562944
 20.868656  16.94207   12.118619   4.3105

[ 7.8438635 16.508146   6.2169685  7.2010436 10.611355  11.455864
 10.141639  10.650343   8.634011  12.256916   5.7028627  6.631567
  8.940851  10.775277   0.3528824  6.379919 ]
[ 10.500029   34.402092  129.32018   108.948326   -3.3940086  47.978245
 144.97197    43.549164   12.774998   31.491     -35.196125   -3.2284336
  20.702042   32.44559   -17.432388  -40.209354 ]
[ 7.4919844 11.946235   2.7204545 -0.3034253  6.023026   5.552863
  9.767791   9.72823    6.696103   5.3889427  2.3673592  6.8708153
  6.409178   7.554325   3.5573149  4.503238 ]
[10.498148   14.7305355  -2.5920641  -0.34213734 10.710646   10.291527
 -0.99051     8.151984    8.4126215  11.232627    9.105264   13.651982
  8.984386    8.396383   -0.4686365  12.652381  ]
[10.616459   13.417256   -1.7334256  -0.94909215  9.302456    8.600305
  0.77234554  9.012844    8.543653    9.840729   10.490463   13.589451
  8.707783    8.123335    3.2230773  13.047472  ]
[ 9.630749   12.497114    1.928626   -0.44465542  7.0447063   6.

[10.9583645  17.122433    6.6734457   7.7165804  11.297575   11.360308
 10.553901   10.697055   12.557241   12.47234     5.77168     6.2724285
  8.923724   10.30415     0.87462664  6.5066667 ]
[12.8991165 18.061428  14.353662   9.888489   8.376734  14.462448
 22.611126  13.788078  16.222109  12.521608   8.415129   6.518918
 10.118401  12.229978   2.6209915  8.693277 ]
[15.321134   17.304012   15.240582   12.913336   11.402277   10.393259
 20.272717   12.271391   14.276722    9.399818    0.12012532  6.4127874
 11.264424   13.167227    9.237898   -2.123612  ]
[14.158776  17.306404  23.254623  19.728283  11.269102  12.763456
 28.496902  13.686918  16.723082  12.9079895  4.8394027  6.832553
 15.409007  18.706974  13.370377   2.8552344]
[17.536894  22.033173  26.95111   33.501213  20.103508  19.389767
 25.825962  20.804756  12.935921  22.605145   6.6756854 13.46222
 23.015224  24.699827  18.155796   8.5358305]
[ 4.215159  18.778238  13.994667  16.179428  18.601385  11.036835
 13.455989  10.

[ 8.659346  16.223555   6.740071   7.8533897 12.712997  12.908081
  5.9149003 10.707571   9.978851  12.506632   5.8850064  6.549114
  9.259155  10.540005   0.7341871  7.5909805]
[  3.6557975  50.8052    195.96439   149.07747     6.684436   73.73839
 208.61967    41.924995   -4.3375573  10.174025  -79.81915   -18.160954
  31.276173   24.448046  -94.07016   -68.72142  ]
[ 9.056996  13.452322  14.167317  11.321565   8.924694  10.508329
 12.012914  12.133131   9.341669  11.118511   1.4863311  4.568193
  7.724429  11.74806    2.610622   5.094365 ]
[ 14.107391   12.5211115   4.355715    7.097271   15.493199   15.645901
 -23.51165     9.752612    5.4680305  18.525719    6.0123563  12.364726
   6.345625   10.630831    6.763038   11.315399 ]
[ 16.729336   12.885027   10.64511    10.33752    12.687473   20.13035
 -29.87865    13.495684    6.565771   26.124775   15.798639   15.218105
   6.7848525  12.50038     9.452965   21.420883 ]
[ 15.507864  13.827158   8.210045   9.040291  12.238558  16.9930

[ 9.839159   17.472061    6.467103    7.766444   12.196517    8.937454
  9.676202   10.852722    9.595125   12.772001    5.8857985   6.26933
  9.637138   10.698137    0.93826246  7.9579124 ]
[ 7.6280794 14.598011  11.516973   3.7750244  5.9252357 10.911977
 19.69857   11.121321   8.356037   7.1425543  0.5293524  4.896577
  6.5795674  7.0102654  5.775826   2.552454 ]
[22.741474  26.138622  35.28341   42.21494   17.834848  18.814022
 36.06614   25.891582  13.040487  21.814854   1.3743765  7.9651017
 19.11482   20.042692  -6.087871  11.876346 ]
[13.382272  16.780529   9.813311   5.5855055 10.123164   8.477169
 19.48229   12.574611  13.649284   6.607013   5.134991   7.128678
 10.22463    7.5332713  8.153915   8.325442 ]
[22.040651  27.654116  26.823355  32.23998   20.808413  17.678505
 29.036243  22.160406  19.017633  16.468086   4.60683    7.854495
 16.560583  14.497541   2.5374756  7.7984843]
[15.796753  19.97851   11.113035   9.7967825 12.325775  15.791199
 20.463408  13.601067  21.6280

In [81]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED é SCALE
seed_value = np.random.randint(1000)
#seed_value = 41
traffic_scale = 0.2


#LAUNCH
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value), '--scale', str(traffic_scale), '--gui-settings-file', 'full_state_sumo_gui_settings.xml']

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        #action = epsilon_greedy_policy(state,0)*2

        action = epsilon_greedy_policy(state, 0)
        feu_1 = action // 4

        feu_2 = action % 4

        print(feu_1, feu_2)
        traci.trafficlight.setPhase(trafic_light_ids[0],2*feu_1)
        traci.trafficlight.setPhase(trafic_light_ids[1],2*feu_2)

    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
[ 7.869684  16.125425   5.8837585  6.8611517 10.878611  10.80106
  8.43807   10.521591   9.312567  12.409721   5.7825227  5.639542
  8.264942   9.425619   0.7756138  6.1479177]
0 1
[ 7.72301   12.424446   1.1650336 -1.3201954  7.7808046  7.9002028
  7.5048094  9.473507   8.716415   8.867413   5.8992705  7.335644
  6.597601   7.196883   4.337574   7.2419786]
0 1
[ 8.9101305 12.004208  -3.6145515 -2.7474418  9.282086   7.2568846
  1.3550138  8.56129    8.5287075 10.382938   8.921721   8.885347
  6.8337803  7.4987526  6.2041407  9.807993 ]
0 1
[ 8.369993   11.635524   -1.9041286  -0.43861294 10.735094    6.299881
  1.523077    7.7854023   6.305703   10.710299    4.6020775   8.172459
  6.4806724   7.4712114   8.128612    3.2451382 ]
0 1
[11.080881  12.375981  -4.9972405 -1.0749581 11.29932    9.096304
 -1.3398046  9.131567   9.00287   15.181919  15.185171  11.487752
  8.334781   9.703384  10.3585415 13.210812 ]
2 2
[ 9.870019  10.907483   1.7268522  1.2158942  7.8086

X Fatal error.
